In [1]:
# A DUAL ASCENT APPROACH FOR STEINER TREE PROBLEMS ON A DIRECTED GRAPH

In [ ]:
# Read and save dataframe
import pandas as pd 
def read_df(file_name):
    df = pd.read_csv("../df/"+file_name+".csv")
    return df

def save_df(df, file_name):
    df.to_csv("../df/"+file_name+".csv", index=False)

In [ ]:
import numpy as np
def read_graph(name):
    with open(name) as f:
        lines = f.readlines()
        arcs = []
        for line in lines:
            if line == '\n': 
                continue
            parts = line.split()
            det = parts[0]
            if det == 'Name':
                name = parts[1]
            elif det == 'Nodes':
                n_vertices = int(parts[1])
            elif det == 'Edges':
                n_edges = int(parts[1])
            elif det == 'E':
                i = int(parts[1])
                j = int(parts[2])
                c = int(parts[3])
                arcij = ((i,j),c)
                arcji = ((j,i),c)
                arcs.append(arcij)
                arcs.append(arcji)
            elif det == 'Terminals':
                n_terminals = int(parts[1])
        vertices = np.arange(1, int(n_vertices)+1)
        vertices = vertices.tolist()
        terminals = np.arange(1, int(n_terminals)+1)
        terminals = terminals.tolist()
        assert(int(n_edges) == len(arcs)/2)
    f.close()
    ### The format of graphs is D=(V,A,R)
    return [vertices, arcs, terminals]

In [ ]:
sizes=["I080/", "I160/"]

In [ ]:
import os
graphs = {}
path = "../ds/"
for size in sizes:
    files = os.listdir(path+size)
    for file in files:
        file_name = file[:-4]
        graph = read_graph(path+size+file)
        graphs[file_name] = graph

In [ ]:
import gurobipy as gp
from gurobipy import GRB
def formulation_3_ILP(graph):
    # set define
    N, E, V = graph
    root = V[0]
    S = [each for each in N if (each not in V)]
    V = V[1:]
    # delete all the arcs that enter the source vertex
    E = [arc for arc in E if not arc[0][1] == root]
    # create the tuple dictionary of arcs
    E_dict = gp.tupledict(E)
    
    # model creation
    m = gp.Model("Steiner_formulation_3")
    
    E_dict_keys = E_dict.keys()
    X_dict = []
    for k in V:
        for arc in E_dict_keys:
            X_dict.append((arc[0], arc[1], k))
    
    # add variables
    x = m.addVars(X_dict,lb=0,vtype=GRB.INTEGER, name='x') # size: |V| * |E|
    y = m.addVars(E_dict_keys,vtype=GRB.INTEGER, name='y') # size: |E|
    
    # set objective value 2.1
    m.setObjective(gp.quicksum(E_dict[i, j] * y[i, j] for (i, j) in E_dict_keys), GRB.MINIMIZE)
    
    # add constraints
    for i in N:
        for k in V:
            # constraint 2.2
            if i == root:
                m.addConstr(x.sum(i,'*',k) - x.sum('*',i,k) == 1)
            elif i == k:
                m.addConstr(x.sum(i,'*',k) - x.sum('*',i,k) == -1)
            else:
                m.addConstr(x.sum(i,'*',k) - x.sum('*',i,k) == 0)
    
    # constraint 2.3
    for i,j,k in X_dict:
        m.addConstr(x[i,j,k] <= y[i,j])
        
    # update the model
    m.update()
    
    # optimize the model
    m.optimize()
    
    # save the optimal solution
    opt_cost = m.objVal
    
    opt_edges = []
    opt_vertices = []
    
    for v in m.getVars():
        # save the vertices
        if v.varName.startswith('y') and v.x != 0:
            opt_vertices.append((v.varName[2:-1], v.x))
                
    opt_runtime = m.Runtime
    
    return opt_vertices, opt_cost, opt_runtime

def formulation_3_LP(graph):
    # set define
    N, E, V = graph
    root = V[0]
    S = [each for each in N if (each not in V)]
    V = V[1:]
    # delete all the arcs that enter the source vertex
    E = [arc for arc in E if not arc[0][1] == root]
    # create the tuple dictionary of arcs
    E_dict = gp.tupledict(E)
    
    # model creation
    m = gp.Model("Steiner_formulation_3")
    
    E_dict_keys = E_dict.keys()
    X_dict = []
    for k in V:
        for arc in E_dict_keys:
            X_dict.append((arc[0], arc[1], k))
    
    # add variables
    x = m.addVars(X_dict,lb=0,vtype=GRB.CONTINUOUS, name='x') # size: |V| * |E|
    y = m.addVars(E_dict_keys,vtype=GRB.CONTINUOUS, name='y') # size: |E|
    
    # set objective value 2.1
    m.setObjective(gp.quicksum(E_dict[i, j] * y[i, j] for (i, j) in E_dict_keys), GRB.MINIMIZE)
    
    # add constraints
    for i in N:
        for k in V:
            # constraint 2.2
            if i == root:
                m.addConstr(x.sum(i,'*',k) - x.sum('*',i,k) == 1)
            elif i == k:
                m.addConstr(x.sum(i,'*',k) - x.sum('*',i,k) == -1)
            else:
                m.addConstr(x.sum(i,'*',k) - x.sum('*',i,k) == 0)
    
    # constraint 2.3
    for i,j,k in X_dict:
        m.addConstr(x[i,j,k] <= y[i,j])
        
    # update the model
    m.update()
    
    # optimize the model
    m.optimize()
    
    # save the optimal solution
    opt_cost = m.objVal
    
    opt_edges = []
    opt_vertices = []
    
    for v in m.getVars():
        # save the vertices
        if v.varName.startswith('y') and v.x != 0:
            opt_vertices.append((v.varName[2:-1], v.x))
                
    opt_runtime = m.Runtime
    
    return opt_vertices, opt_cost, opt_runtime

In [ ]:
# Log file format:
# ILP runtime
# ILP cost
# LP runtime
# LP cost
# Whether ILP cost is different from LP cost
# new line
# ILP solution
# new line
# LP solution
def record(file_name):
    graph = graphs[file_name]
    ilp_v, ilp_c, ilp_r = formulation_3_ILP(graph)
    lp_v, lp_c, lp_r = formulation_3_LP(graph)
    f = open("../log/"+file_name+"_log.txt", "wt")
    f.write(str(ilp_r) + "\n")
    f.write(str(ilp_c) + "\n")
    f.write(str(lp_r) + "\n")
    f.write(str(lp_c) + "\n")
    f.write(str(lp_c == ilp_c))
    f.write("\n")
    for each in ilp_v:
        f.write(str(each)+ "\n")
    f.write("\n")
    for each in lp_v:
        f.write(str(each)+ "\n")

In [ ]:
for each in graphs.keys():
    record(each)